# Get EFD DATA

This notebook illustrates how to performs simple queries to the influxDB EFD.

Before running this notebook make sure to install the following libraries:

Open a terminal and run 

    - pip install aioinflux
    - pip install datashader

In [1]:
import matplotlib

%matplotlib inline
from matplotlib import pylab as plt
import aioinflux
import getpass
import pandas as pd
import asyncio

from datetime import datetime
import datashader as ds
import datashader.transfer_functions as tf

from bokeh.models import BoxZoomTool, DatetimeTickFormatter
from datashader.bokeh_ext import InteractiveImage
import bokeh.plotting as bp
bp.output_notebook()

Loading BokehJS ...

In [2]:
# Cloud deployment
# INFLUXDB_API_URL = "https://influxdb-efd.lsst.codes"

# Lab deployment
# INFLUXDB_API_URL = "https://test-influxdb-efd.lsst.codes"

# Summit deployment
INFLUXDB_API_URL = 'summit-influxdb-efd.lsst.codes'

INFLUXDB_DATABASE = "efd"

In [3]:
username = "saluser"
password = getpass.getpass(prompt=f'Password for user `{username}`:')

Password for user `saluser`: ········


In [4]:
client = aioinflux.InfluxDBClient(host=INFLUXDB_API_URL, 
                                  port='443', 
                                  ssl=True, 
                                  username=username, 
                                  password=password,
                                  db=INFLUXDB_DATABASE)

In [5]:
client.output = 'dataframe'

timestamp_start = 1576286396
timestamp_window_size = 10.
time_str_beg = datetime.fromtimestamp(timestamp_start).strftime('%Y-%m-%dT%H:%M:%SZ')
time_str_end = datetime.fromtimestamp(timestamp_start+timestamp_window_size).strftime('%Y-%m-%dT%H:%M:%SZ')
print(time_str_beg, time_str_end)
time_span = f"time >= '{time_str_beg}' AND time <= '{time_str_end}'"
print(time_span)

2019-12-14T01:19:56Z 2019-12-14T01:20:06Z
time >= '2019-12-14T01:19:56Z' AND time <= '2019-12-14T01:20:06Z'


In [6]:
async def get_data_frame(field_base, topic, els=100):
    """Method to get high frequency data. 
    """
    fields = ", ".join([f'"{field_base}{i}"' for i in range(100)])
    df = await client.query(f'SELECT "cRIO_timestamp", "private_sndStamp", {fields} FROM "efd"."autogen"."{topic}" WHERE {time_span}')

    times = []
    timestamps = []
    vals = []
    step = 1./els
    for row in df.itertuples():
        for i in range(els):
            times.append(row.cRIO_timestamp + i*step)
            timestamps.append((pd.Timestamp(row.cRIO_timestamp, unit='s', tz='GMT') + pd.Timedelta(i*step, unit='s')))
            vals.append(getattr(row, f'{field_base}{i}'))

    return pd.DataFrame({'times':times, field_base:vals}, index=timestamps)

This next cell will give high frequency (100Hz) data. 

In [ ]:
# tel_elevation = await get_data_frame("elevationCalculatedAngle","lsst.sal.ATMCS.mount_AzEl_Encoders")
# tel_azimuth = await get_data_frame("elevationCalculatedAngle","lsst.sal.ATMCS.mount_AzEl_Encoders")

In [7]:
tel_pos = await client.query(f'SELECT "cRIO_timestamp", "elevationCalculatedAngle99", "azimuthCalculatedAngle99" FROM "efd"."autogen"."lsst.sal.ATMCS.mount_AzEl_Encoders" WHERE {time_span}')

In [8]:
tel_pos

,cRIO_timestamp,elevationCalculatedAngle99,azimuthCalculatedAngle99
2019-12-14 01:19:56.424431470+00:00,1.576286e+09,52.414001,132.459990
2019-12-14 01:19:57.425414122+00:00,1.576286e+09,52.416606,132.461032
2019-12-14 01:19:59.634881163+00:00,1.576286e+09,52.419298,132.462084
2019-12-14 01:19:59.846039564+00:00,1.576286e+09,52.422090,132.463143
2019-12-14 01:20:00.374002872+00:00,1.576286e+09,52.424658,132.464196
2019-12-14 01:20:01.383911567+00:00,1.576286e+09,52.427210,132.465271
2019-12-14 01:20:02.365507895+00:00,1.576286e+09,52.429844,132.466325
2019-12-14 01:20:03.398657084+00:00,1.576286e+09,52.432567,132.467372
2019-12-14 01:20:04.434147100+00:00,1.576286e+09,52.435271,132.468430
2019-12-14 01:20:05.317347549+00:00,1.576286e+09,52.437943,132.469514


In [9]:
a = {"a": 1, "b": 2}

In [14]:
list(a)

['a', 'b']

In [17]:
import types

In [18]:
check = types.SimpleNamespace()

In [37]:
types.SimpleNamespace(**dict(zip(list(a), [True]*len(a))))

namespace(a=True, b=True)

In [28]:
for i in zip(list(a), [True]*len(a)):
    print(i)

('a', True)
('b', True)


In [36]:
**dict(zip(list(a), [True]*len(a)))

SyntaxError: invalid syntax (<ipython-input-36-d24397ff358c>, line 1)